In [14]:
from ndreg import *
import matplotlib
import ndio.remote.neurodata as neurodata
import numpy as np
import cv2

## Mean shift clustering function

In [7]:
import numpy as np

# seeds: set of seeds (described in Section 2.2.4)
# points: set of voxels whose intensity exceeds the background threshold
# radius: is the radius used for the kernel
# bandwidth: is a value that allows us to change the convergence threshold
def cluster(seeds, points, radius, bandwidth = 1):
    Z = 0
    for point in points:
        Z += point[3]

    C_set = set()
    
    
    
    num_seeds = len(seeds)
    seed_iter = 0
    for seed in seeds: 
        converged = False

        while not converged:
            prev_seed = seed
            new_seed = np.zeros(len(seed))
            for point in points:
                new_seed += (point[3] * point * spherical_kernel(prev_seed - point, radius)) / Z
            converged = check_converged(new_seed, prev_seed, bandwidth)
        C_set.add(new_seed)
    return C_set


    # for p in seeds:
    #     c = p
    #     converged = False
    #     # not sure if implemented correctly?
    #     while not converged:
    #         prev = c
    #         c = np.zeros(len(c))
    #         for q in points:
    #             c += (intensities[q] * q * spherical_kernel(prev - q, radius)) / Z
    #         converged = check_converged(c, prev, bandwidth)
    #     C_set.add(c)
    # return C_set


# used to see if a voxel has converged yet
# voxel_1 is the voxel after the most recent update
# voxel_2 is the voxel prior to the most recent update
# bandwidth allows us to change how we want our data to converge
def check_converged(voxel_1, voxel_2, bandwidth):
    dist = np.linalg.norm(voxel_1 - voxel_2)
    if abs(dist) < .001 * bandwidth:    
        return True
    return False                        


# a is the voxel
# R is a parameter that should be smaller than the expected radius of a cell
def spherical_kernel(a, R):
    if np.linalg.norm(a) < R:
        return 1
    return 0

## File path checking function

In [29]:
import os
import errno

def make_sure_path_exists(path):
    """Check if the directory a file is going to be written to exists, and if not, create the directory."""
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise

## loading csv function

In [10]:
def loadCsv(path):
    """Method for getting a numpy array from the csv file"""
    points = []
    with open(path, 'r') as infile:
        for line in infile:
            line = line.strip().split(',')
            entry = [int(line[0]), int(line[1]), int(line[2]), int(line[3])]
            points.append(entry)
    points = np.array(points)
    return points

## generating plotly function

In [45]:
import plotly
import plotly.graph_objs as graphobjs

def csv_to_plot(file_path, token, resolution, outfile_name=""):
    """Generates the plotly from the csv file."""
    # Type in the path to your csv file here
    thedata = None
    thedata = np.genfromtxt(file_path, delimiter=',', dtype='int', usecols = (0,1,2), names=['a','b','c'])
    print(thedata)

    # Set tupleResolution to resolution input parameter
    tupleResolution = resolution;

    # EG: for Aut1367, the spacing is (0.01872, 0.01872, 0.005).
    xResolution = tupleResolution[0]
    yResolution = tupleResolution[1]
    zResolution = tupleResolution[2]
    # Now, to get the mm image size, we can multiply all x, y, z
    # to get the proper mm size when plotting.
    
    x = [x for x in thedata['a']]
    print('x:')
    print(x)

    trace1 = graphobjs.Scatter3d(
        x = [x * xResolution for x in thedata['a']],
        y = [x * yResolution for x in thedata['b']],
        z = [x * zResolution for x in thedata['c']],
        mode='markers',
        marker=dict(
            size=1.2,
            color='cyan',                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.15
        )
    )

    data = [trace1]
    layout = graphobjs.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        paper_bgcolor='rgb(0,0,0)',
        plot_bgcolor='rgb(0,0,0)'
    )

    fig = graphobjs.Figure(data=data, layout=layout)
#     print(self._token + "plotly")

    make_sure_path_exists('plots')
    
    if outfile_name == "":
        plotly.offline.plot(fig, filename= 'plots/' + token + "_plot.html")
    else:
        plotly.offline.plot(fig, filename= 'plots/' + outfile_name + "_plot.html")

In [34]:
import plotly
import plotly.graph_objs as graphobjs

def array_to_plot(points, token, resolution, outfile_name=""):
    """Generates the plotly from the csv file."""
    # Type in the path to your csv file here
    
    thedata = points

    # Set tupleResolution to resolution input parameter
    tupleResolution = resolution;

    # EG: for Aut1367, the spacing is (0.01872, 0.01872, 0.005).
    xResolution = tupleResolution[0]
    yResolution = tupleResolution[1]
    zResolution = tupleResolution[2]
    # Now, to get the mm image size, we can multiply all x, y, z
    # to get the proper mm size when plotting.

    trace1 = graphobjs.Scatter3d(
        x = [x * xResolution for x in thedata['a']],
        y = [x * yResolution for x in thedata['b']],
        z = [x * zResolution for x in thedata['c']],
        mode='markers',
        marker=dict(
            size=1.2,
            color='cyan',                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.15
        )
    )

    data = [trace1]
    layout = graphobjs.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        paper_bgcolor='rgb(0,0,0)',
        plot_bgcolor='rgb(0,0,0)'
    )

    fig = graphobjs.Figure(data=data, layout=layout)
#     print(self._token + "plotly")

    make_sure_path_exists('plots')
    
    if outfile_name == "":
        plotly.offline.plot(fig, filename= 'plots/' + token + "_plot.html")
    else:
        plotly.offline.plot(fig, filename= 'plots/' + outfile_name + "_plot.html")

## Getting initial points

In [36]:
inToken = 'Fear199'

csv_file_path = 'points/' + inToken + '.csv'

points = loadCsv(csv_file_path)

print('shape:')
print(points.shape)
print(points)

shape:
(7467, 4)
[[  0   6 309 252]
 [  0  16 460 219]
 [  0 110 184 234]
 ..., 
 [382 118 368 249]
 [382 179 414 240]
 [382 216 505 249]]


## Getting Seeds

In [12]:
seeds = points[points[:, 3] > 250]

print('seeds shape:')
print(seeds.shape)

print('seeds:')
print(seeds)

seeds shape:
(2043, 4)
seeds:
[[  0  29 296 252]
 [  0  91  65 255]
 [  1  44 337 255]
 ..., 
 [382 211 499 252]
 [382 226 318 252]
 [382 262 144 252]]


## Performing Mean Shift Clustering

In [15]:
inImg = imgDownload(inToken, resolution=5)    # store downsampled level 5 brain to memory

In [16]:
resolution = inImg.GetSpacing();

In [33]:
clustered_points = cluster(seeds, points, radius=20, bandwidth = 1)

KeyboardInterrupt: 

## Plotting original points

In [47]:
token = 'Fear199'
file_path = 'points/' + token + ".csv"
print(resolution)
csv_to_plot(file_path, token, resolution, "Fear199_og")

(0.01872, 0.01872, 0.005)
[(0, 6, 309) (0, 16, 460) (0, 110, 184) ..., (382, 118, 368)
 (382, 179, 414) (382, 216, 505)]
x:
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20,

In [46]:
token = 'Fear199'
file_path = 'points/' + token + ".csv"
print(resolution)
array_to_plot(file_path, token, resolution, "Fear199_og")

(0.01872, 0.01872, 0.005)


TypeError: string indices must be integers, not str

In [6]:
import numpy as np

x = [0, 0, 0]
y = [1, 1, 1]

xnp = np.array(x)
ynp = np.array(y)

arr = np.vstack((xnp, ynp))

print(arr)

print(arr[0])

for row in arr:
    print(row)

[[0 0 0]
 [1 1 1]]
[0 0 0]
[0 0 0]
[1 1 1]
